In [3]:
!pip install ace-tools yfinance

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     -------------------- ---------------- 524.3/948.2 kB 16.4 MB/s eta 0:00:01
     -------------------------------------- 948.2/948.2 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.8-py3-none-any.whl size=139064 sha256=accb54b90e3cf48807c19d1f591e4db492cc1dbe9302f4d18c8ff139a8a011a6
  Stored in directory: c:\users\tommc\appdata\local\pip\cache\wheels\8f\65\34\456800445efeafb05164fe95285c70e81ba1d96bae30f43917
Successfully built peewee


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import yfinance as yf
import pandas as pd
import time
import os
from datetime import datetime, timedelta

TECH_STOCKS = [
    "AAPL", "MSFT", "NVDA", "AVGO", "CRM", "ORCL", "CSCO", "ACN", "NOW", "IBM",
    "AMD", "ADBE", "INTU", "QCOM", "TXN", "AMAT", "PLTR", "ANET", "PANW", "MU",
    "ADI", "LRCX", "KLAC", "APH", "INTC", "CDNS", "CRWD", "MSI", "SNPS", "ADSK",
    "FTNT", "ROP", "WDAY", "NXPI", "FICO", "TEL", "IT", "CTSH", "GLW", "DELL",
    "HPQ", "MCHP", "ANSS", "MPWR", "HPE", "KEYS", "GDDY", "CDW", "TYL", "NTAP"
]

MACRO_TICKERS = {
    '^VIX': 'Volatility Index',
    '^TNX': '10-Year Treasury Yield',
    '^FVX': '5-Year Treasury Yield',
    'CL=F': 'Crude Oil Futures'
}

DATA_FOLDER = "yfinance_tech_stock_data"
MACRO_FOLDER = "macroeconomic_data"
os.makedirs(DATA_FOLDER, exist_ok=True)
os.makedirs(MACRO_FOLDER, exist_ok=True)

def fetch_stock_data(symbol, interval="1h", period="2y"):
    """Fetch and save stock data with enhanced error handling"""
    try:
        print(f"\n📈 Fetching {symbol} ({interval} interval)...")
        df = yf.Ticker(symbol).history(period=period, interval=interval)
        
        if df.empty:
            print(f"🚨 No data for {symbol}")
            return None

        df = df.reset_index().rename(columns={"Datetime": "timestamp"})
        df['symbol'] = symbol  
        

        path = os.path.join(DATA_FOLDER, f"{symbol}_stock.csv")
        df.to_csv(path, index=False)
        print(f"✅ Saved {len(df)} rows to {path}")
        
        return df

    except Exception as e:
        print(f"🚨 Error fetching {symbol}: {str(e)}")
        return None

def fetch_macro_data():
    """Fetch macroeconomic indicators with retry logic"""
    macro_data = {}
    
    for ticker, name in MACRO_TICKERS.items():
        retries = 3
        while retries > 0:
            try:
                print(f"\n🌐 Fetching {name} ({ticker})...")
                df = yf.download(ticker, start=datetime.now()-timedelta(days=730), interval='1h')
                
                if not df.empty:
                    df = df.reset_index().rename(columns={'Date': 'timestamp'})
                    df['ticker'] = ticker
                    macro_data[ticker] = df
                    

                    path = os.path.join(MACRO_FOLDER, f"{ticker.replace('^','')}.csv")
                    df.to_csv(path, index=False)
                    print(f"✅ Saved {ticker} data")
                    

                    print(f"\n📋 Columns for {ticker} ({name}):")
                    print(df.columns.tolist())
                    print(f"\nFirst row for {ticker}:")
                    print(df.head(1))
                    break
                else:
                    print(f"🚨 Empty data for {ticker}")
                    retries -= 1
                    
            except Exception as e:
                print(f"🚨 Error ({retries} retries left): {str(e)}")
                retries -= 1
                time.sleep(5)
    
    return macro_data

def main_data_pipeline():
    """Orchestrate data fetching with rate limiting"""
    stock_dfs = []
    preview_printed = False  
    

    for symbol in TECH_STOCKS:
        df = fetch_stock_data(symbol)
        if df is not None:
            stock_dfs.append(df)
            if not preview_printed:
                print(f"\n📋 Columns for {symbol}:")
                print(df.columns.tolist())
                print(f"\nFirst row for {symbol}:")
                print(df.head(1))
                preview_printed = True
        time.sleep(1.5)
    

    macro_data = fetch_macro_data()
    
    return pd.concat(stock_dfs), macro_data


if __name__ == "__main__":
    stock_data, macro_data = main_data_pipeline()
    print("\n🎉 Data pipeline complete!")
    print(f"Stock data shape: {stock_data.shape}")
    print(f"Macro data keys: {list(macro_data.keys())}")


📈 Fetching AAPL (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data\AAPL_stock.csv

📋 Columns for AAPL:
['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'symbol']

First row for AAPL:
                  timestamp        Open    High         Low       Close  \
0 2023-01-31 09:30:00-05:00  142.699997  143.75  142.279999  143.169998   

     Volume  Dividends  Stock Splits symbol  
0  13897777        0.0           0.0   AAPL  

📈 Fetching MSFT (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data\MSFT_stock.csv

📈 Fetching NVDA (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data\NVDA_stock.csv

📈 Fetching AVGO (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data\AVGO_stock.csv

📈 Fetching CRM (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data\CRM_stock.csv

📈 Fetching ORCL (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data\ORCL_stock.csv

📈 Fetching CSCO (1h interval)...
✅ Saved 3494 rows 

[*********************100%***********************]  1 of 1 completed


✅ Saved ^VIX data

📋 Columns for ^VIX (Volatility Index):
[('Datetime', ''), ('Close', '^VIX'), ('High', '^VIX'), ('Low', '^VIX'), ('Open', '^VIX'), ('Volume', '^VIX'), ('ticker', '')]

First row for ^VIX:
Price                   Datetime      Close   High        Low   Open Volume  \
Ticker                                 ^VIX   ^VIX       ^VIX   ^VIX   ^VIX   
0      2023-02-02 08:00:00+00:00  17.620001  17.75  17.610001  17.74      0   

Price  ticker  
Ticker         
0        ^VIX  

🌐 Fetching 10-Year Treasury Yield (^TNX)...


[*********************100%***********************]  1 of 1 completed


✅ Saved ^TNX data

📋 Columns for ^TNX (10-Year Treasury Yield):
[('Datetime', ''), ('Close', '^TNX'), ('High', '^TNX'), ('Low', '^TNX'), ('Open', '^TNX'), ('Volume', '^TNX'), ('ticker', '')]

First row for ^TNX:
Price                   Datetime  Close   High    Low   Open Volume ticker
Ticker                             ^TNX   ^TNX   ^TNX   ^TNX   ^TNX       
0      2023-02-02 13:20:00+00:00  3.344  3.378  3.344  3.373      0   ^TNX

🌐 Fetching 5-Year Treasury Yield (^FVX)...


[*********************100%***********************]  1 of 1 completed


✅ Saved ^FVX data

📋 Columns for ^FVX (5-Year Treasury Yield):
[('Datetime', ''), ('Close', '^FVX'), ('High', '^FVX'), ('Low', '^FVX'), ('Open', '^FVX'), ('Volume', '^FVX'), ('ticker', '')]

First row for ^FVX:
Price                   Datetime  Close   High    Low   Open Volume ticker
Ticker                             ^FVX   ^FVX   ^FVX   ^FVX   ^FVX       
0      2023-02-02 13:20:00+00:00  3.423  3.462  3.423  3.454      0   ^FVX

🌐 Fetching Crude Oil Futures (CL=F)...


[*********************100%***********************]  1 of 1 completed


✅ Saved CL=F data

📋 Columns for CL=F (Crude Oil Futures):
[('Datetime', ''), ('Close', 'CL=F'), ('High', 'CL=F'), ('Low', 'CL=F'), ('Open', 'CL=F'), ('Volume', 'CL=F'), ('ticker', '')]

First row for CL=F:
Price                   Datetime      Close       High        Low       Open  \
Ticker                                 CL=F       CL=F       CL=F       CL=F   
0      2023-02-02 04:00:00+00:00  77.220001  77.239998  77.050003  77.099998   

Price  Volume ticker  
Ticker   CL=F         
0           0   CL=F  

🎉 Data pipeline complete!
Stock data shape: (174699, 9)
Macro data keys: ['^VIX', '^TNX', '^FVX', 'CL=F']


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import logging
import re
from datetime import datetime, timedelta
from typing import List, Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

TECH_STOCKS = [
    "AAPL", "MSFT", "NVDA", "AVGO", "CRM", "ORCL", "CSCO", "ACN", "NOW", "IBM",
    "AMD", "ADBE", "INTU", "QCOM", "TXN", "AMAT", "PLTR", "ANET", "PANW", "MU",
    "ADI", "LRCX", "KLAC", "APH", "INTC", "CDNS", "CRWD", "MSI", "SNPS", "ADSK",
    "FTNT", "ROP", "WDAY", "NXPI", "FICO", "TEL", "IT", "CTSH", "GLW", "DELL",
    "HPQ", "MCHP", "ANSS", "MPWR", "HPE", "KEYS", "GDDY", "CDW", "TYL", "NTAP"
]

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ProgressTracker:
    def __init__(self, total_months: int):
        self.start_time = time.time()
        self.total_months = total_months
        self.completed_months = 0
        self.total_articles = 0
        self.current_month = None
        
    def update_progress(self):
        elapsed = time.time() - self.start_time
        avg_time = elapsed / (self.completed_months + 1e-6)
        remaining = avg_time * (self.total_months - self.completed_months)
        
        progress = (
            f"\n📅 Current: {self.current_month} | "
            f"Completed: {self.completed_months}/{self.total_months} months | "
            f"Articles: {self.total_articles} | "
            f"Elapsed: {timedelta(seconds=int(elapsed))} | "
            f"ETA: {timedelta(seconds=int(remaining))}"
        )
        sys.stdout.write("\r\033[K" + progress)
        sys.stdout.flush()

class VoxScraper:
    def __init__(self, start_date: datetime, end_date: datetime):
        self.base_url = "https://www.vox.com"
        self.rate_limiter = RateLimiter(calls=3, period=1)
        self.article_buffer = []
        self.all_articles = []  # Stores all articles across months
        self.start_date = start_date
        self.end_date = end_date
        self.total_months = ((end_date.year - start_date.year) * 12 
                            + end_date.month - start_date.month + 1)
        self.progress = ProgressTracker(self.total_months)
        self.financial_terms = re.compile(
            r'\b(stock|market|tech|economy|investment|trading|'
            r'earnings|valuation|merger|acquisition|IPO)\b', 
            re.IGNORECASE
        )

    def fetch_article_content(self, url: str) -> Dict:
        with self.rate_limiter:
            try:
                response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'html.parser')

                time_tag = soup.find('time')
                article_date = pd.to_datetime(time_tag['datetime']) if time_tag else None

                article_body = soup.find('div', class_='c-entry-content')
                text_content = ' '.join([p.get_text() for p in article_body.find_all('p')]) if article_body else ''

                self.progress.total_articles += 1
                self.progress.update_progress()

                return {
                    'url': url,
                    'timestamp': article_date,
                    'content': text_content,
                    'is_financial': bool(self.financial_terms.search(text_content)),
                    'tickers': self._extract_tickers(text_content)
                }

            except Exception as e:
                logging.error(f"Failed to fetch {url}: {str(e)}")
                return None

    def _extract_tickers(self, text: str) -> List[str]:
        return [ticker for ticker in TECH_STOCKS if re.search(r'\b' + ticker + r'\b', text)]

    def scrape_time_range(self):
        current_date = self.start_date
        while current_date <= self.end_date:
            year = current_date.year
            month = current_date.month
            self.progress.current_month = f"{year}-{month:02d}"
            self.progress.update_progress()
            
            self._scrape_month(year, month)
            current_date += timedelta(days=32)
            self.progress.completed_months += 1
            self.progress.update_progress()

    def _scrape_month(self, year: int, month: int):
        page = 1
        while True:
            url = f"{self.base_url}/archives/{year}/{month}/{page}"
            try:
                article_links = self._get_article_links(url)
                if not article_links:
                    break
                
                with ThreadPoolExecutor(max_workers=4) as executor:
                    futures = [executor.submit(self.fetch_article_content, link) for link in article_links]
                    for future in as_completed(futures):
                        if (result := future.result()) is not None:
                            self.article_buffer.append(result)
                
                page += 1
                time.sleep(0.5)
                
            except Exception as e:
                logging.error(f"Stopping scrape for {year}-{month:02d}: {str(e)}")
                break
                
        self._save_buffer(year, month)

    def _get_article_links(self, url: str) -> List[str]:
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            return [
                self.base_url + a['href'] 
                for a in soup.select('div._1p9ghgz3 a[href^="/"]')
                if not a['href'].startswith("/archives")
            ]
        except Exception as e:
            logging.error(f"Failed to get links from {url}: {str(e)}")
            return []

    def _save_buffer(self, year: int, month: int):
        if not self.article_buffer:
            return
            
        df = pd.DataFrame(self.article_buffer)
        self.all_articles.extend(self.article_buffer) 
        
        df['date_hour'] = df['timestamp'].dt.floor('h')
        df = df.dropna(subset=['date_hour'])
        
        path = f"vox_articles_{year}_{month:02d}.parquet"
        df.to_parquet(path, index=False)
        logging.info(f"\n✅ Saved {len(df)} articles to {path}")
        self.article_buffer.clear()

class RateLimiter:
    def __init__(self, calls: int, period: int):
        self.calls = calls
        self.period = period
        self.timestamps = []
        
    def __enter__(self):
        now = time.time()
        self.timestamps = [t for t in self.timestamps if t > now - self.period]
        
        if len(self.timestamps) >= self.calls:
            sleep_time = self.period - (now - self.timestamps[0])
            time.sleep(sleep_time)
            
        self.timestamps.append(time.time())
        
    def __exit__(self, *args):
        pass

if __name__ == "__main__":
    start_date = datetime(2023, 1, 1)
    end_date = datetime.now()
    
    scraper = VoxScraper(start_date, end_date)
    print("🚀 Starting Vox.com scraper with progress tracking:")
    print(f"⏳ Time range: {start_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}")
    print("📊 Progress will be updated in real-time below:\n")
    
    try:
        scraper.scrape_time_range()

        final_df = pd.DataFrame(scraper.all_articles)
        
        print("\n\n🎉 Final Report:")
        print(f"Total articles collected: {len(final_df)}")
        
        if not final_df.empty:
            stock_articles = final_df[final_df['tickers'].apply(len) > 0]
            print(f"\n📈 Tech Stock Articles Found ({len(stock_articles)}):")
            for idx, row in stock_articles.iterrows():
                print(f"\n📰 Article {idx + 1}:")
                print(f"   URL: {row['url']}")
                print(f"   Date: {row['timestamp'].strftime('%Y-%m-%d') if pd.notnull(row['timestamp']) else 'Unknown'}")
                print(f"   Tickers: {', '.join(row['tickers'])}")
                print(f"   Financial Terms Found: {'Yes' if row['is_financial'] else 'No'}")
            print(f"\n💾 Full data saved to vox_articles_YYYY_MM.parquet files")
        else:
            print("\nNo articles found in the specified date range")
            
        print(f"\nProcessed months: {scraper.progress.completed_months}/{scraper.total_months}")
        
    except KeyboardInterrupt:
        print("\n\n⚠️ Scraping interrupted by user! Partial results saved.")

🚀 Starting Vox.com scraper with progress tracking:
⏳ Time range: 2023-01 to 2025-02
📊 Progress will be updated in real-time below:


Current: 2023-01 | Completed: 0/26 months | Articles: 0 | Elapsed: 0:00:00 | ETA: 7:17:50
Current: 2023-01 | Completed: 0/26 months | Articles: 1 | Elapsed: 0:00:04 | ETA: 1356 days, 9:07:52
Current: 2023-01 | Completed: 0/26 months | Articles: 2 | Elapsed: 0:00:04 | ETA: 1400 days, 0:21:18
Current: 2023-01 | Completed: 0/26 months | Articles: 3 | Elapsed: 0:00:04 | ETA: 1409 days, 8:15:06
Current: 2023-01 | Completed: 0/26 months | Articles: 4 | Elapsed: 0:00:05 | ETA: 1602 days, 16:24:30
Current: 2023-01 | Completed: 0/26 months | Articles: 5 | Elapsed: 0:00:06 | ETA: 1883 days, 21:42:58
Current: 2023-01 | Completed: 0/26 months | Articles: 6 | Elapsed: 0:00:06 | ETA: 1903 days, 3:55:13
Current: 2023-01 | Completed: 0/26 months | Articles: 7 | Elapsed: 0:00:06 | ETA: 1924 days, 19:55:14
Current: 2023-01 | Completed: 0/26 months | Articles: 8 | Elapsed: 

2025-02-01 10:11:21,969 - INFO - 
✅ Saved 177 articles to vox_articles_2023_01.parquet



Current: 2023-01 | Completed: 1/26 months | Articles: 177 | Elapsed: 0:02:16 | ETA: 0:56:53
Current: 2023-02 | Completed: 1/26 months | Articles: 177 | Elapsed: 0:02:16 | ETA: 0:56:53
Current: 2023-02 | Completed: 1/26 months | Articles: 178 | Elapsed: 0:02:20 | ETA: 0:58:20
Current: 2023-02 | Completed: 1/26 months | Articles: 179 | Elapsed: 0:02:20 | ETA: 0:58:26
Current: 2023-02 | Completed: 1/26 months | Articles: 180 | Elapsed: 0:02:20 | ETA: 0:58:38
Current: 2023-02 | Completed: 1/26 months | Articles: 181 | Elapsed: 0:02:21 | ETA: 0:58:47
Current: 2023-02 | Completed: 1/26 months | Articles: 182 | Elapsed: 0:02:21 | ETA: 0:59:04
Current: 2023-02 | Completed: 1/26 months | Articles: 183 | Elapsed: 0:02:22 | ETA: 0:59:15
Current: 2023-02 | Completed: 1/26 months | Articles: 184 | Elapsed: 0:02:22 | ETA: 0:59:18
Current: 2023-02 | Completed: 1/26 months | Articles: 185 | Elapsed: 0:02:22 | ETA: 0:59:25
Current: 2023-02 | Completed: 1/26 months | Articles: 186 | Elapsed: 0:02:23 | 

2025-02-01 10:14:01,118 - INFO - 
✅ Saved 188 articles to vox_articles_2023_02.parquet



Current: 2023-02 | Completed: 2/26 months | Articles: 365 | Elapsed: 0:04:55 | ETA: 0:59:08
Current: 2023-03 | Completed: 2/26 months | Articles: 365 | Elapsed: 0:04:55 | ETA: 0:59:08
Current: 2023-03 | Completed: 2/26 months | Articles: 366 | Elapsed: 0:04:59 | ETA: 0:59:56
Current: 2023-03 | Completed: 2/26 months | Articles: 367 | Elapsed: 0:05:00 | ETA: 1:00:01
Current: 2023-03 | Completed: 2/26 months | Articles: 368 | Elapsed: 0:05:00 | ETA: 1:00:04
Current: 2023-03 | Completed: 2/26 months | Articles: 369 | Elapsed: 0:05:01 | ETA: 1:00:14
Current: 2023-03 | Completed: 2/26 months | Articles: 370 | Elapsed: 0:05:01 | ETA: 1:00:17
Current: 2023-03 | Completed: 2/26 months | Articles: 371 | Elapsed: 0:05:01 | ETA: 1:00:20
Current: 2023-03 | Completed: 2/26 months | Articles: 372 | Elapsed: 0:05:01 | ETA: 1:00:23
Current: 2023-03 | Completed: 2/26 months | Articles: 373 | Elapsed: 0:05:02 | ETA: 1:00:30
Current: 2023-03 | Completed: 2/26 months | Articles: 374 | Elapsed: 0:05:03 | 

...


Current: 2024-11 | Completed: 22/26 months | Articles: 4070 | Elapsed: 0:53:06 | ETA: 0:09:39
Current: 2024-12 | Completed: 22/26 months | Articles: 4070 | Elapsed: 0:53:06 | ETA: 0:09:39
Current: 2024-12 | Completed: 22/26 months | Articles: 4071 | Elapsed: 0:53:09 | ETA: 0:09:39
Current: 2024-12 | Completed: 22/26 months | Articles: 4072 | Elapsed: 0:53:10 | ETA: 0:09:40
Current: 2024-12 | Completed: 22/26 months | Articles: 4073 | Elapsed: 0:53:10 | ETA: 0:09:40
Current: 2024-12 | Completed: 22/26 months | Articles: 4074 | Elapsed: 0:53:10 | ETA: 0:09:40
Current: 2024-12 | Completed: 22/26 months | Articles: 4075 | Elapsed: 0:53:11 | ETA: 0:09:40
Current: 2024-12 | Completed: 22/26 months | Articles: 4076 | Elapsed: 0:53:11 | ETA: 0:09:40
Current: 2024-12 | Completed: 22/26 months | Articles: 4077 | Elapsed: 0:53:12 | ETA: 0:09:40
Current: 2024-12 | Completed: 22/26 months | Articles: 4078 | Elapsed: 0:53:12 | ETA: 0:09:40
Current: 2024-12 | Completed: 22/26 months | Articles: 4079

2025-02-01 11:03:57,256 - INFO - 
✅ Saved 155 articles to vox_articles_2024_12.parquet



Current: 2024-12 | Completed: 23/26 months | Articles: 4226 | Elapsed: 0:54:51 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4226 | Elapsed: 0:54:51 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4227 | Elapsed: 0:54:53 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4228 | Elapsed: 0:54:54 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4229 | Elapsed: 0:54:54 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4230 | Elapsed: 0:54:54 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4231 | Elapsed: 0:54:55 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4232 | Elapsed: 0:54:55 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4233 | Elapsed: 0:54:56 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4234 | Elapsed: 0:54:56 | ETA: 0:07:09
Current: 2025-01 | Completed: 23/26 months | Articles: 4235

2025-02-01 11:06:12,520 - INFO - 
✅ Saved 194 articles to vox_articles_2025_01.parquet



📅 Current: 2025-01 | Completed: 24/26 months | Articles: 4420 | Elapsed: 0:57:07 | ETA: 0:04:45

🎉 Final Report:
Total articles collected: 4420

📈 Tech Stock Articles Found (0):

💾 Full data saved to vox_articles_YYYY_MM.parquet files

Processed months: 24/26
